# Causality mining using OpenAI API


## Loading packages and data

In [159]:
from openai import OpenAI
from tqdm import tqdm
import pandas as pd
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
import re

In [160]:
who_data = pd.read_csv("../data/corpus.csv")
who_data_epi = who_data[who_data["InformationType"] == "Epidemiology"]
who_data_assessment = who_data[who_data["InformationType"] == "Assessment"]
who_data_epi_and_assessment = who_data[who_data["InformationType"].isin(["Epidemiology", "Assessment"])]

## Function defnition

### Prompt Generation

In [161]:
class PromptDesigner:
    def __init__(self):
        # Store different parts of the prompt as class attributes
        self.persona_task_description = """
        You are an epidemiologist tasked with identifying sentences or phrases from outbreak reports that describe the drivers or contributors to the emergence or transmission of emerging pests and pathogens.
        """

        self.domain_localization = """
        Here is the definition of DPSIR (Drivers, Pressure, State, Impacts, and Responses) framework, where it shows how drivers are associated with the emergence of disease.
        Drivers: underlying socio-economic, environmental, or ecological forces that create conditions favourable for how a disease emerges, spreads or sustains transmission in human, animals or plants.
        Pressure: human anthropogenic activities that are mainly responsible for the chances of spillover events and the transmission of pests and pathogens.
        State: the current circulation of pests and pathogens, represented as either new case detected, an endemic, an epidemic or a pandemic.
        Impacts: the effects caused by pests and pathogens on individuals, communities' socio-economic, and political.
        Responses: the actions and interventions taken by governments to manage the occurrence of drivers and pressures, and to control the spread of pests and pathogens and to mitigate the impacts.
        """

        self.causality_definition = """
        Causality definition: In the reports, causality can take two forms. The first form is "Intra-sentence causality", where the “cause” and the “effect” lie in a single sentence, while in "Inter-sentence causality", the “cause” and the “effect” lie in different sentences.
        """
        
        self.extraction_guide = """
        Input text: The sudden appearance of unlinked cases of mpox in South Africa without a history of international travel, the high HIV prevalence among confirmed cases, and the high case fatality ratio suggest that community transmission is underway, and the cases detected to date represent a small proportion of all mpox cases that might be occurring in the community; it is unknown how long the virus may have been circulating. This may in part be due to the lack of early clinical recognition of an infection with which South Africa previously gained little experience during the ongoing global outbreak, potential pauci-symptomatic manifestation of the disease, or delays in care-seeking behaviour due to limited access to care or fear of stigma.
        
        Expected output
        1. Raw text with marked causes and effects
        The sudden appearance of unlinked cases of mpox in South Africa without a history of international travel, the high HIV prevalence among confirmed cases, and the high case fatality ratio suggest that (E1) community transmission (E1) is underway, and the cases detected to date represent a small proportion of all mpox cases that might be occurring in the community; it is unknown how long the virus may have been circulating. This may in part be due to the (C1) lack of early clinical recognition of an infection (C1) with which South Africa previously gained little experience during the ongoing global outbreak, potential (C1) pauci-symptomatic manifestation of the disease (C1), or (C1, E2) delays in care-seeking behavior (C1, E2) due to (C2) limited access to care (C2) or (C2) fear of stigma (C2).
       
        2. Extracted causes and effects
        C1: lack of early clinical recognition of an infection -> E1: community transmission 
        C1: pauci-symptomatic manifestation of the disease -> E1: community transmission 
        C1: delays in care-seeking behavior -> E1: community transmission 
        C2: limited access to care -> E2: delays in care-seeking behaviour
        C2: fear of stigma -> -> E2: delays in care-seeking behaviour delays in care-seeking behaviour  
        """

        self.few_shot_examples = """
        Below are some examples how causality can be reported in different forms:
        - Single cause, single effect (Type 1)

        Example 1: (C1) High population density and mobility in urban areas (C1) have facilitated (E1) the rapid spread of the virus (E1)". 

        Example 2: There is (C1) no vaccine for Influenza A(H1N1)v infection currently licensed for use in humans (C1). Seasonal influenza vaccines against human influenza viruses are generally not expected to protect people from (E1) infection with influenza viruses (E1) that normally circulate in pigs, but they can reduce severity.


        - Single cause, multiple effects (Type 2)

        Example 3: Several countries including Cameroon, Ethiopia, Haiti, Lebanon, Nigeria (north-east of the country), Pakistan, Somalia, Syria and the Democratic Republic of Congo (eastern part of the country) are in the midst of complex (C1) humanitarian crises (C1) with (E1) fragile health systems (E1), (E1) inadequate access to clean water and sanitation (E1) and have (E1) insufficient capacity to respond to the outbreaks (E1)

        - Multiple causes, single effect (Type 3)
        Example 4: Moreover, (C1) a low index of suspicion (C1), (C1) socio-cultural norms (C1), (C1) community resistance (C1), (C1) limited community knowledge regarding anthrax transmission (C1), (C1) high levels of poverty (C1) and (C1) food insecurity (C1), (C1) a shortage of available vaccines and laboratory reagents (C1), (C1) inadequate carcass disposal (C1) and (C1) decontamination practices (C1) significantly contribute to hampering (E1) the containment of the anthrax outbreak (E1).

        Example 5:
        The (E1) risk at the national level (E1) is assessed as 'High' due to the following:
        + In other parts of Timor-Leste (C1) health workers have limited knowledge dog bite and scratch case management (C1) including PEP and RIG administration
        + (C2) Insufficient stock of human rabies vaccines (C2) in the government health facilities.

        - Multiple causes, multiple effects (Type 4) - Chain of causalities
        The text may describe a chain of causality, where one effect becomes then the cause of another effect. To describe the chain, you should number the causes and effects. For example, cause 1 (C1) -> effect 1 (E1), but since effect 1 is also cause of effect 2, you should do cause 1 (C1) -> effect 1 (E1, C2) -> effect 2 (E2). 

        Example 6: (E2) The risk of insufficient control capacities (E2) is considered high in Zambia due to (C1) concurrent public health emergencies in the country (cholera, measles, COVID-19) (C1) that limit the country’s human and (E1, C2) financial capacities to respond to the current anthrax outbreak adequately (E1, C2).

        Example 7: (C1) Surveillance systems specifically targeting endemic transmission of chikungunya or Zika are weak or non-existent (C1) -> (E1, C2) Misdiagnosis between diseases  & Skewed surveillance (E1, C2) -> (E2, C3) Misinform policy decisions (E2, C3) -> (E3)reduced accuracy on the estimation of the true burden of each diseases (E3), poor risk assessments (E3), and non optimal clinical management and resource allocation (E3). 

        Example 8: (C1) Changes in the predominant circulating serotype (C1) -> (E1, C2) increase the population risk of subsequent exposure to a heterologous DENV serotype (E1, C2), -> (E2) which increases the risk of higher rates of severe dengue and deaths (E2).

        """

        self.negative_cases = """
        Irrelevant causality (negative cases): Some sentences contain causal relationships, but the effect may not be related to the disease transmission or emergence. Avoid classifying those causal relationships.

        Example 1 (no causality): Because these viruses continue to be detected in swine populations worldwide, further human cases following direct or indirect contact with infected swine can be expected.

        Example 2 (no relevant causality): There is some (E1) pressure on the healthcare capacity (E1) due to the (C1) very high number of admissions for dengue (C1); (C1) high vector density (C1); and an (C1) anticipated prolonged monsoon (C1). 

        Example 3 (no relevant causality): (C1) MVD is a highly virulent disease (C1) that can cause (E1) haemorrhagic fever (E1) and is clinically similar to Ebola virus disease.

        """

        self.mechanism_of_causality = """
        When the text describes/list possible mechanisms behind the cause of transmission or emergence, tag them with (M). A mechanism of causality describes the specific interactions between the pathogen, host, and environment that causes the transmission / emergence. They often describe interactions at the physiological level. 

        Example 1: The global outbreak 2022 — 2024 has shown that (C1) sexual contact (C1) enables faster and more efficient (E1) spread of the virus (E1) from one person  to another due to (M1) direct contact of mucous membranes between people (M1), (M1) contact with multiple partners (M1), (M1) a possibly shorter incubation period on average (M1), and (M1) a longer infectious period for immunocompromised individuals (M1).

        """

        self.sign_of_causality = """
        For each cause-effect relationship, indicate whether each cause (C) is positive (C+) or negative (C-) and each effect (E) is positive (E+) or negative (E-). 
        Use the list of positive and negative sign words provided to help determine the sign of each cause and effect. Be mindful of sentences with negations (e.g., “does not improve”), which reverses polarity. 
        Positive sign words: increase, facilitate, support, improve, expand, promote, enable, enhance, accelerate, advance, grow, boost, strengthen, benefit, contribute, progress, initiate, develop, elevate, stimulate, alleviate, optimize, revitalize. 
        Negative sign words: limit, decrease, reduce, hamper, hinder, restrict, suppress, impair, inhibit, undermine, challenge, disrupt, lack, insufficient, incomplete, challenge, deficit, obstacle, barrier, diminish, shortage, scarcity, obstruct, worsen, decline. 

        Example 1: “(C1-) a lack of timely access to diagnostics in many areas (C1-), (C1-) incomplete epidemiological investigations (C1-), (C1-) challenges in contact tracing and extensive but inconclusive animal investigations (C1-) continue to hamper rapid response (E1-)”

        Example 2: Moreover, (C1-) a low index of suspicion (C1-), (C1) socio-cultural norms (C1), (C1) community resistance (C1), (C1-) limited community knowledge regarding anthrax transmission (C1-), (C1+) high levels of poverty (C1+) and (C1) food insecurity (C1), (C1-) a shortage of available vaccines and laboratory reagents (C1-), (C1-) inadequate carcass disposal (C1-) and (C1) decontamination practices (C1) significantly contribute to hampering (E1-) the containment of the anthrax outbreak (E1-).
        """

    def generate_prompt(self, include_persona=False, include_domain=False, include_causality=False, include_guidance = False, include_examples=False, include_negative=False, include_mechanism=False, include_sign=False):
        """
        Dynamically generate a prompt based on the specified parts.
        """
        # Start with an empty prompt
        prompt = ""

        # Append parts based on the arguments provided
        if include_persona:
            prompt += self.persona_task_description + "\n"
        
        if include_domain:
            prompt += self.domain_localization + "\n"
        if include_causality:
            prompt += self.causality_definition + "\n"
        if include_guidance:
            prompt += self.extraction_guide + "\n"
        if include_examples:
            prompt += self.few_shot_examples + "\n"
        if include_negative:
            prompt += self.negative_cases + "\n"
        if include_mechanism:
            prompt += self.mechanism_of_causality + "\n"
        if include_sign:
            prompt += self.sign_of_causality + "\n"

        return prompt

# prompt_designer = PromptDesigner()
# prompt_with_mechanism = prompt_designer.generate_prompt(
#     include_persona=True, 
#     include_domain=True, 
#     include_examples=True, 
#     include_mechanism=True
# )
# print(prompt_with_mechanism)

### Causality Extraction

In [162]:
# Read API key from the .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Function to split text into chunks
def batch(iterable, n=1):
    """Utility function to batch sentences into chunks."""
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

class CausalChain:
    def __init__(self, dataframe, prompt_designer=None):
        self.dataframe = dataframe
        self.outlines = []  # Store a list of dictionaries to represent complex relationships
        self.prompt_designer = prompt_designer if prompt_designer else PromptDesigner()
        self.processed_chunks = set()  # Track processed chunks to avoid repetition

    def create_effects(self, batch_size=16, prompt_parts={}):
        print("Analyzing causation...")

        with open("api_responses.txt", "w", encoding="utf-8") as file:
            for index, row in tqdm(self.dataframe.iterrows(), total=self.dataframe.shape[0]):
                text = row['Text']
                don_id = row['DonId']

                # Split text into sentences and then into chunks of 3 sentences
                sentences = text.split(". ")
                chunks = [". ".join(a) + "." for a in batch(sentences, 3)]

                for chunk in chunks:
                    # Skip if the chunk has already been processed
                    if (don_id, chunk) in self.processed_chunks:
                        continue

                    cause_effect_pairs, raw_texts, causality_types, response_text = self.extract_cause_effect_openai(chunk, prompt_parts, don_id)

                    # Mark the chunk as processed
                    self.processed_chunks.add((don_id, chunk))

                    # Write the response to the file immediately after receiving it
                    file.write(f"\n\nAPI Response for Article ID {don_id}: \n{response_text}\n\n")

                    if not cause_effect_pairs:
                        print(f"No cause-effect pairs found for chunk: {chunk}")

                    for pair, raw_text, causality_type in zip(cause_effect_pairs, raw_texts, causality_types):
                        cause, effect = pair
                        self.outlines.append({
                            "DonId": don_id,
                            "Cause": cause,
                            "Effect": effect,
                            "Causality_Type": causality_type,
                            "Raw_Text": raw_text
                        })

        # Print the raw texts, causes, effects, and types of causality
        if self.outlines:
            for outline in self.outlines:
                print(f"DonId: {outline['DonId']}")
                print(f"Raw Text: {outline['Raw_Text']}")
                print(f"Cause: {outline['Cause']}")
                print(f"Effect: {outline['Effect']}")
                print(f"Causality Type: {outline['Causality_Type']}")
                print("\n")
        else:
            print("No cause-effect pairs found in the entire dataset.")

    def extract_cause_effect_openai(self, chunk, prompt_parts={}, don_id=None):
        # Use the PromptDesigner to generate the customized prompt
        prompt = self.prompt_designer.generate_prompt(**prompt_parts)

        # Append the text chunk to the prompt and provide a clear format for the response
        full_prompt = f"""{prompt}

        Input text: {chunk}

        Expected output format:
        1. Raw text with marked causes and effects:
        [Provide the input text with marked causes and effects]

        2. Extracted causes and effects:
        C1: [cause] -> E1: [effect], Causality type: [T1/T2/...]
        C2: [cause] -> E2: [effect], Causality type: [T1/T2/...]
        ...
        """

        # Call the OpenAI API
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": full_prompt}],
            max_tokens=1000,
            temperature=0,
        )

        response_text = response.choices[0].message.content
        print(f"API Response for Article ID {don_id}: {response_text}")  # Print the API response for debugging
        return self.parse_response(response_text) + (response_text,)

    @staticmethod
    def parse_response(response_text):
        cause_effect_pairs = []
        raw_texts = []
        causality_types = []  # Store the causality types

        if not response_text.strip():
            print("Empty response received from API.")
            return cause_effect_pairs, raw_texts, causality_types

        # Parse the response based on the expected output format
        lines = response_text.split("\n")
        raw_text_section = False
        extracted_pairs_section = False
        raw_text = ""

        for line in lines:
            line = line.strip()

            if line.startswith("1. Raw text with marked causes and effects"):
                raw_text_section = True
                extracted_pairs_section = False
                raw_text = ""  # Reset raw text for each new section
                continue

            if line.startswith("2. Extracted causes and effects"):
                raw_text_section = False
                extracted_pairs_section = True
                continue

            if raw_text_section and line:
                raw_text += line + " "

            if extracted_pairs_section and line.startswith("C") and "->" in line:
                try:
                    cause = line.split(":")[1].split("->")[0].strip()
                    effect = line.split("->")[1].split(", Causality type:")[0].strip()
                    causality_type = line.split("Causality type:")[1].strip()
                    cause_effect_pairs.append((cause, effect))
                    raw_texts.append(raw_text.strip())
                    causality_types.append(causality_type)
                except IndexError:
                    print(f"Malformed line: {line}")

        return cause_effect_pairs, raw_texts, causality_types


def create_causes_effects_dataframe(outlines):
    # Create a DataFrame from the outlines list
    dataframe = pd.DataFrame(outlines)

    # Ensure all expected columns are present, fill with 'Unknown' if missing
    for column in ["DonId", "Cause", "Effect", "Causality_Type", "Raw_Text"]:
        if column not in dataframe:
            dataframe[column] = "Unknown"

    return dataframe

# Function to edit .txt file
def format_api_responses(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    # Ensure each "API Response for Article ID" starts on a new line
    # Add a line break before "API Response for Article ID" if it's directly after some text
    formatted_content = re.sub(r"(.*?)API Response for Article ID", r"\1\n\nAPI Response for Article ID", content)

    # Write the formatted content back to a new file
    with open("formatted_api_responses.txt", "w", encoding="utf-8") as output_file:
        output_file.write(formatted_content)


### Drivers Categorization

- Allow testing with zero shot, one shot, few shots example
- Allow for different AI services
- Print raw text
- Allow for classification -> later steps
- Run a loop through the whole dataframe
- Allow for cateforisation of sign of the causality
- Allow for validation
- Have one “conversation” per document, only put instructions at the start

## Experimenting the model

In [163]:
example_data = who_data_assessment.iloc[1:6]

In [164]:
prompt_designer = PromptDesigner()

prompt_parts = {
    "include_persona": True,
    "include_domain": True,
    "include_causality": True,
    "include_guidance": True,
    "include_examples": True,
    "include_negative": True,
    "include_mechanism": False,
    "include_sign": False,
}

# Create a CausalChain instance with the dataset
causal_chain = CausalChain(dataframe=example_data, prompt_designer=prompt_designer)

# Generate effects based on the chunks of text
causal_chain.create_effects(prompt_parts=prompt_parts)

Analyzing causation...


  0%|                                                    | 0/2 [00:00<?, ?it/s]

API Response for Article ID 2024-DON538: 1. Raw text with marked causes and effects:
Although no cases of WNV have been documented in birds or horses in the country, it is possible that the virus is circulating in these populations undetected. Despite this, the overall impact on public health remains limited at this stage, as there is currently only one recorded human case and appropriate public health response measures have been implemented, as described above. The risk of international dissemination of WNV from Barbados is low. The virus is primarily transmitted through (C1) mosquito bites (C1), with (C2) birds as the natural hosts (C2). There is no evidence to suggest that WNV spreads easily between humans or from horses to mosquitoes.

2. Extracted causes and effects:
C1: mosquito bites -> E1: transmission of the virus, Causality type: T1
C2: birds as the natural hosts -> E1: transmission of the virus, Causality type: T1


 50%|██████████████████████                      | 1/2 [00:04<00:04,  4.06s/it]

API Response for Article ID 2024-DON538: 1. Raw text with marked causes and effects:
Therefore, the potential for widespread international transmission remains low. Based on these criteria, the overall risk related to the public health remains low. WHO will continue to evaluate the epidemiological situation in Barbados.

2. Extracted causes and effects:
No relevant causality related to the emergence or transmission of pests and pathogens is present in the input text.
No cause-effect pairs found for chunk: Therefore, the potential for widespread international
transmission remains low.&nbsp;Based
on these criteria, the overall risk related to the public health remains low.
WHO will continue to evaluate the epidemiological situation in Barbados.&nbsp;.
API Response for Article ID 2024-DON536: 1. Raw text with marked causes and effects:
The notification of this case does not change the overall risk assessment. The new case reported is believed to have acquired (E1) MERS-CoV infection (E1) 

100%|████████████████████████████████████████████| 2/2 [00:13<00:00,  6.85s/it]

API Response for Article ID 2024-DON536: 1. Raw text with marked causes and effects:
Human-to-human transmission of MERS-CoV may occur if there are (C1) delays in identifying the infection (C1), particularly in (C2) countries that are not well-acquainted with the disease (C2), as well as (C1) slow triage of suspected cases (C1) and (C1) delays in the implementation of standard infection prevention and control measures (C1). WHO continues to monitor the epidemiological situation and conducts risk assessments based on the latest available information.

2. Extracted causes and effects:
C1: delays in identifying the infection -> E1: Human-to-human transmission of MERS-CoV, Causality type: T3
C1: slow triage of suspected cases -> E1: Human-to-human transmission of MERS-CoV, Causality type: T3
C1: delays in the implementation of standard infection prevention and control measures -> E1: Human-to-human transmission of MERS-CoV, Causality type: T3
C2: countries that are not well-acquainted with

In [165]:
# Create a DataFrame with the causes, effects, and other related information
result_df = create_causes_effects_dataframe(causal_chain.outlines)

# Display the resulting DataFrame
print(result_df)
# format_api_responses("api_responses.txt")

          DonId                                              Cause                                        Effect Causality_Type                                           Raw_Text
0   2024-DON538                                     mosquito bites                 E1: transmission of the virus             T1  Although no cases of WNV have been documented ...
1   2024-DON538                         birds as the natural hosts                 E1: transmission of the virus             T1  Although no cases of WNV have been documented ...
2   2024-DON536                                 locally within KSA                        E1: MERS-CoV infection             T1  The notification of this case does not change ...
3   2024-DON536                        individual visited Pakistan  E1: potential for international transmission             T3  However, the (E1) potential for international ...
4   2024-DON536  high-risk contact traveled to South Asia withi...  E1: potential for international trans